### <font color='blue'> COLETA E PREPARAÇÃO DOS DADOS </font>

In [1]:
!ls -lh ./datasets

total 4.6M
-rw-r--r-- 1 1000 1000 3.9M Aug 25 21:09 census.csv
-rw-r--r-- 1 1000 1000 117K Aug 15 16:45 credit-data.csv
-rw-r--r-- 1 1000 1000 344K Sep 21 01:08 dataset_Unicarioca.xlsx
-rw-r--r-- 1 1000 1000  26K Sep  6 01:17 dataset_Unicarioca_v1.xlsx
-rw-r--r-- 1 1000 1000 203K Sep 16 18:43 dataset_Unicarioca_v2.xlsx
-rw-r--r-- 1 1000 1000  538 Aug 28 16:54 risco-credito.csv


In [2]:
from pandas import ExcelFile, read_excel, DataFrame, to_numeric, set_option
from numpy import int8, NaN
from unidecode import unidecode
import hashlib
# ---------------------------------
import warnings; warnings.filterwarnings('ignore')
# set_option('display.max_rows', dataframe.T.count().size)
# set_option('display.max_columns', dataframe.columns.size)
# ---------------------------------

input_dataset = './datasets/dataset_Unicarioca.xlsx'
sheet_names = ExcelFile(input_dataset).sheet_names[1:] # [1:9] [9:]  [26:] 
dataframe = DataFrame()

for sheet_name in sheet_names:
    data = read_excel(input_dataset, sheet_name)
    # --------------------------------- 
    if (data.iloc[[4],[6]].values[0] == ['AV2']):
        lcolumns1 = list(data.iloc[[4],[3,6,9]].values[0]) + list(data.iloc[[3],[14]].values[0])
        dataframe_part = data.iloc[5:,[1,2,3,6,9,14]].values
    else:
        if (list(data.iloc[[3],[15]].values[0]) == ['FALTAS']):
            lcolumns1 = list(data.iloc[[4],[3,8,11]].values[0]) + list(data.iloc[[3],[15]].values[0])
            dataframe_part = data.iloc[5:,[1,2,3,8,11,15]].values
        else:
            lcolumns1 = list(data.iloc[[4],[3,8,11]].values[0]) + list(data.iloc[[3],[16]].values[0])
            dataframe_part = data.iloc[5:,[1,2,3,8,11,16]].values
    
    ## Remover acentos
    listColums1 = [unidecode(ii) for ii in list(data.iloc[[3],[1,2]].values[0]) + lcolumns1 ]
    # --------------------------------- 
    listColums2 = []
    listColums2.append(data.columns[0][:-1]) # Curso
    listColums2.append(data.columns[2:4].values[0][0:-1]) # Período
    listColums2.append(data[0:1].values[0][0][:-1]) # Turma
    listColums2.append(data[0:1].values[0][2][:-1]) # Turno
    listColums2.append(data[1:2].values[0][0][:-1]) # Disciplina
    listColums2.append(data[2:3].values[0][0][:-1]) # Professor
    listColums2 = [ unidecode(ii) for ii in listColums2 ] # Remover acentos
    # --------------------------------- 
    ## Primeira parte do Dataset
    dataframe_part = DataFrame(dataframe_part, columns=listColums1)
    dataframe_part = DataFrame(dataframe_part, columns= (listColums2 + listColums1) )
    # --------------------------------- 
    ## Preencher segunda Parte
    dataframe_part[ listColums2[0] ] = data.columns[1] # Curso
    dataframe_part[ listColums2[1] ] = str(data.columns[2:4].values[1])[0:7] # Período
    dataframe_part[ listColums2[2] ] = str(data[0:1].values[0][1])+'-'+str(data.columns[2:4].values[1])[0:7] # Turma
    dataframe_part[ listColums2[3] ] = unidecode(data[0:1].values[0][3]) # Turno
    dataframe_part[ listColums2[4] ] = unidecode(data[1:2].values[0][1]) # Disciplina
    dataframe_part[ listColums2[5] ] = data[2:3].values[0][1] # Professor
    # --------------------------------- 
    dataframe = dataframe.append(dataframe_part)

## Restaurar Index
dataframe.index = range(dataframe.shape[0]) 

## Trocar preenchimento '-' por zeros
for ii in ['AV1','AV2','AV3']:
    dataframe[ii].replace({0:'-'}, inplace=True) # Trocar zero por '-' para trocar virgula por ponto
    dataframe[ii] = [ jj.replace(',', '.') for jj in dataframe[ii].values ] # Trocar vírgila por ponto
    dataframe[ii].replace({'-':0}, inplace=True)
    dataframe[ii] = dataframe[ii].apply(to_numeric)

dataframe.FALTAS.replace({'-':0}, inplace=True)
dataframe.FALTAS = dataframe.FALTAS.astype(int8)

## Remover colunas
dataframe.drop(['Professor','Nome'], axis=1, inplace=True)

## Tratar coluna Matricula (md5, sha1, sha256, sha512)
dataframe.Matricula = [ hashlib.md5(str(ii).encode('utf-8')).hexdigest() for ii in dataframe.Matricula ]

## Trocar espaçoes em branco por "underscore"
dataframe.Disciplina = [ ii.replace(' ', '_') for ii in  dataframe.Disciplina ]


## Salvar em .csv
dataframe.to_csv('datasetCommon.csv', sep=';', index=False)

In [3]:
print(dataframe.shape)
dataframe.head()

(2897, 10)


,Curso,Periodo,Turma,Turno,Disciplina,Matricula,AV1,AV2,AV3,FALTAS
0,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,74629a22cfed47eae86cd7e71c2d1635,6.5,8.0,0.0,2
1,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,435b15978f23ed38232820320e38f3b2,3.5,7.8,6.5,5
2,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,a1602a9569fa3532f0a3833eeb72daff,7.5,3.0,6.5,5
3,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,2a5fd0f356429b54e90a586ae9b81ab8,0.0,0.0,0.0,11
4,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,a107e3dbdd1692a3e8903aaf95335e38,5.0,6.0,5.3,5


### <font color='blue'> Feature Engineering </font>

In [4]:
from pandas import read_csv

dataframe = read_csv('./datasetCommon.csv', sep=';')

## Adicionar Colunas
dataframe = DataFrame(dataframe, columns=(list(dataframe.columns[:-2]) +
                                ['MEDIAParcial'] + 
                                [dataframe.columns[-2]] + ['MEDIAFinal'] +
                                [dataframe.columns[-1]] + ['AV3Bin']))

## Alimentar Colunas
dataframe.MEDIAParcial = round((dataframe.AV1 + dataframe.AV2)/2, 1)
dataframe.MEDIAFinal = [ round(((dataframe.AV1.values[ii] + dataframe.AV2.values[ii] + dataframe.AV3.values[ii]) - 
                             min(dataframe.AV1.values[ii], dataframe.AV2.values[ii], dataframe.AV3.values[ii]))/2, 1) 
                     for ii in range(len(dataframe)) ]

## Binarizar coluna AV3
dataframe.AV3Bin = dataframe.AV3
# --------------------------------- 
dataframe.loc[ (dataframe.AV3Bin > 0) , ['AV3Bin'] ] = 1
# --------------------------------- 
# from numpy import where
# dataframe.AV3Bin = where(dataframe.AV3Bin == 0, 0, 1 )


## Salvar em .csv
dataframe.to_csv('datasetFull.csv', sep=';', index=False)

In [5]:
print(dataframe.shape)
dataframe.head()

(2897, 13)


,Curso,Periodo,Turma,Turno,Disciplina,Matricula,AV1,AV2,MEDIAParcial,AV3,MEDIAFinal,FALTAS,AV3Bin
0,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,74629a22cfed47eae86cd7e71c2d1635,6.5,8.0,7.2,0.0,7.2,2,0.0
1,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,435b15978f23ed38232820320e38f3b2,3.5,7.8,5.6,6.5,7.2,5,1.0
2,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,a1602a9569fa3532f0a3833eeb72daff,7.5,3.0,5.2,6.5,7.0,5,1.0
3,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,2a5fd0f356429b54e90a586ae9b81ab8,0.0,0.0,0.0,0.0,0.0,11,0.0
4,ADM,2019-01,152-2019-01,Noite,GESTAO_DE_SISTEMAS_DE_INFORMACAO,a107e3dbdd1692a3e8903aaf95335e38,5.0,6.0,5.5,5.3,5.6,5,1.0


### <font color='blue'> One Hot Encoding </font>

In [6]:
from pandas import read_csv, concat, get_dummies

dataframe = read_csv('./datasetFull.csv', sep=';')

attribute = []
attribute.append( dataframe.iloc[:,1:3] ) # Primeira parte do dataset
attribute.append( get_dummies( dataframe.Turno ) )
attribute.append( get_dummies( dataframe.Curso ) )
attribute.append( get_dummies( dataframe.Disciplina ) )
attribute.append( dataframe.iloc[:,-8:] ) # Ultima parte do dataset

dataframe = concat(attribute, axis=1)


## Salvar em .csv
dataframe.to_csv('datasetDummy.csv', sep=';', index=False)

In [7]:
print(dataframe.shape)
dataframe.head()

(2897, 31)


,Periodo,Turma,Manha,Noite,ADM,ADS,CIC,DES,ENGC,ANALISE_E_PROJETO_DE_SISTEMAS,...,SIMULACAO_EMPRESARIAL,WEBDESIGN,Matricula,AV1,AV2,MEDIAParcial,AV3,MEDIAFinal,FALTAS,AV3Bin
0,2019-01,152-2019-01,0,1,1,0,0,0,0,0,...,0,0,74629a22cfed47eae86cd7e71c2d1635,6.5,8.0,7.2,0.0,7.2,2,0.0
1,2019-01,152-2019-01,0,1,1,0,0,0,0,0,...,0,0,435b15978f23ed38232820320e38f3b2,3.5,7.8,5.6,6.5,7.2,5,1.0
2,2019-01,152-2019-01,0,1,1,0,0,0,0,0,...,0,0,a1602a9569fa3532f0a3833eeb72daff,7.5,3.0,5.2,6.5,7.0,5,1.0
3,2019-01,152-2019-01,0,1,1,0,0,0,0,0,...,0,0,2a5fd0f356429b54e90a586ae9b81ab8,0.0,0.0,0.0,0.0,0.0,11,0.0
4,2019-01,152-2019-01,0,1,1,0,0,0,0,0,...,0,0,a107e3dbdd1692a3e8903aaf95335e38,5.0,6.0,5.5,5.3,5.6,5,1.0


In [8]:
dataframe.dtypes

Periodo                                               object
Turma                                                 object
Manha                                                  uint8
Noite                                                  uint8
ADM                                                    uint8
ADS                                                    uint8
CIC                                                    uint8
DES                                                    uint8
ENGC                                                   uint8
ANALISE_E_PROJETO_DE_SISTEMAS                          uint8
BANCO_DE_DADOS_I                                       uint8
DESENVOLVIMENTO_DE_APLICACOES_COM_BANCO_DE_DADOS       uint8
DESENVOLVIMENTO_DE_APLICACOES_DISTRIBUIDAS             uint8
DESENVOLVIMENTO_DE_APLICACOES_MOVEIS                   uint8
DESENVOLVIMENTO_DE_APLICACOES_WEB                      uint8
EVOLUCAO_DO_PENSAMENTO_ADMINISTRATIVO                  uint8
GESTAO_DE_SISTEMAS_DE_IN

In [9]:
turno = dataframe.iloc[:,2:4]
curso = dataframe.iloc[:,4:9]
disciplina = dataframe.iloc[:,9:23]

In [10]:
turno

,Manha,Noite
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1
8,0,1
9,0,1


In [11]:
curso

,ADM,ADS,CIC,DES,ENGC
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [12]:
disciplina

,ANALISE_E_PROJETO_DE_SISTEMAS,BANCO_DE_DADOS_I,DESENVOLVIMENTO_DE_APLICACOES_COM_BANCO_DE_DADOS,DESENVOLVIMENTO_DE_APLICACOES_DISTRIBUIDAS,DESENVOLVIMENTO_DE_APLICACOES_MOVEIS,DESENVOLVIMENTO_DE_APLICACOES_WEB,EVOLUCAO_DO_PENSAMENTO_ADMINISTRATIVO,GESTAO_DE_SISTEMAS_DE_INFORMACAO,INTRODUCAO_A_PROGRAMACAO,PROGRAMACAO_ESTRUTURADA,PROJETO_INTEGRADOR_EM_ENGENHARIA_DA_COMPUTACAO_II,PROJETO_INTEGRADO_EM_ENGENHARIA_DA_COMPUTACAO_I,SIMULACAO_EMPRESARIAL,WEBDESIGN
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0,0,0,0,0,0
